In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (TrainingArguments,
                          pipeline,
                          AutoTokenizer,
                          AutoModelForQuestionAnswering,
                          Trainer,
                          logging)
from datasets import Dataset, load_dataset
import evaluate
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/data/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/data/test.csv")

In [5]:
import re
def clean_answer(answer):
    # Remove annotations like <<>> and final '#### ...'
    answer = re.sub(r'<<.*?>>', '', answer)
    answer = re.sub(r'####.*', '', answer)
    return answer.strip()

df_train['clean_answer'] = df_train['answer'].apply(clean_answer)
df_test['clean_answer'] = df_test['answer'].apply(clean_answer)

# Split answers into stepwise reasoning
def split_steps(answer):
    return answer.split('\n')

df_train['steps'] = df_train['clean_answer'].apply(split_steps)
df_test['steps'] = df_test['clean_answer'].apply(split_steps)

# 3. Prepare dataset for BERT input
def create_input_output_pairs(row):
    return {
        'question': row['question'],
        'answer': row['clean_answer']
    }

data_pairs_train = df_train.apply(create_input_output_pairs, axis=1).tolist()
data_pairs_test = df_test.apply(create_input_output_pairs, axis=1).tolist()

print(data_pairs_train[0])
print(data_pairs_test[0])

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = 24 clips in May.\nNatalia sold 48+24 = 72 clips altogether in April and May.'}
{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': 'Janet sells 16 - 3 - 4 = 9 duck eggs a day.\nShe makes 9 * 2 = $18 every day at the farmer’s market.'}


In [6]:
def transform_data(data_list):
    transformed_data = []

    for data in data_list:
        try:
            # Tìm câu hỏi chính
            question_sentences = data["question"].strip().split('.')
            last_question = question_sentences[-1].strip() + '?' if question_sentences[-1] else question_sentences[-2].strip() + '?'

            # Combine remaining question sentences (without the last sentence) with the answer
            combined_context = ". ".join(question_sentences[:-1]).strip()
            if combined_context:
                combined_context += ". "
            combined_context += data["answer"]

            last_answer_sentence = combined_context.strip().split('\n')[-1]

            # Find the start index of the last answer sentence
            start_index = combined_context.rfind(last_answer_sentence)

            # Chuẩn hóa dữ liệu
            transformed_data.append({
                "question": last_question.strip(),
                "context": combined_context.strip().replace('\n', ' '),
                "answers": {
                    "answer_start": [start_index],
                    "text": [last_answer_sentence]
                }
            })
        except Exception as e:
            print(f"Skipping entry due to error: {e}")
            continue

    return transformed_data

dataset_train_transform = transform_data(data_pairs_train)
dataset_test_transform = transform_data(data_pairs_test)
print(dataset_train_transform[0])
print(dataset_test_transform[0])



{'question': 'How many clips did Natalia sell altogether in April and May??', 'context': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. Natalia sold 48/2 = 24 clips in May. Natalia sold 48+24 = 72 clips altogether in April and May.', 'answers': {'answer_start': [132], 'text': ['Natalia sold 48+24 = 72 clips altogether in April and May.']}}
{'question': "How much in dollars does she make every day at the farmers' market??", 'context': "Janet’s ducks lay 16 eggs per day.  She eats three for breakfast every morning and bakes muffins for her friends every day with four.  She sells the remainder at the farmers' market daily for $2 per fresh duck egg. Janet sells 16 - 3 - 4 = 9 duck eggs a day. She makes 9 * 2 = $18 every day at the farmer’s market.", 'answers': {'answer_start': [259], 'text': ['She makes 9 * 2 = $18 every day at the farmer’s market.']}}


In [7]:
# Convert to Hugging Face Datasets
dataset_train = Dataset.from_list(dataset_train_transform)
dataset_test = Dataset.from_list(dataset_test_transform)

print(dataset_train)
print(dataset_test)

Dataset({
    features: ['question', 'context', 'answers'],
    num_rows: 14946
})
Dataset({
    features: ['question', 'context', 'answers'],
    num_rows: 2638
})


In [8]:
# Define model and tokenize

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenize_function(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


# Tokenize datasets
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)


Map:   0%|          | 0/14946 [00:00<?, ? examples/s]

Map:   0%|          | 0/2638 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=500,  # Match the evaluation frequency to save steps
    learning_rate=2e-5,
    warmup_steps=500,  # Warmup for learning rate scheduling
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    num_train_epochs=5,  # Increased epochs for better convergence
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    load_best_model_at_end=True  # Load the best model at the end of training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)
# Train the model
print("\nStarting training...")
trainer.train()

# Save the model
model.save_pretrained("bert_gsm8k_qa")
tokenizer.save_pretrained("bert_gsm8k_qa")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-abddabbd7dad>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



Starting training...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,0.032800,0.061828
1000,0.011800,0.050452
1500,0.021900,0.048497
2000,0.028700,0.039820
2500,0.000300,0.046208
3000,0.040100,0.055423
3500,0.000000,0.051539
4000,0.000100,0.060717
4500,0.000000,0.062641


('bert_gsm8k_qa/tokenizer_config.json',
 'bert_gsm8k_qa/special_tokens_map.json',
 'bert_gsm8k_qa/vocab.txt',
 'bert_gsm8k_qa/added_tokens.json',
 'bert_gsm8k_qa/tokenizer.json')

In [28]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="/content/bert_gsm8k_qa")

# Example
context = 'Hao have five apples. Each people have 2 apples. Hao buys three more 5 + 3 = 8. Hao buys three more and share with four friends.'
question = 'How many apple does each people have??'



# Thực hiện suy luận
result = qa_pipeline(question=question, context=context)

# In kết quả
print(f"Question: {question}")
print(f"Answer: {result['answer']}")
print(f"Score: {result['score']:.2f}")

Device set to use cuda:0


Question: How many apple does each people have??
Answer: Hao buys three more and share with four friends.
Score: 1.00
